In [12]:
from mysql import connector

mysql_connection = connector.connect(
host="localhost",
user="root",
passwd="password",
auth_plugin='mysql_native_password',
database="tmdb_schema",
port="3306") #Initialise the connection to the SQL server with the sql server and schema details

mycursor = mysql_connection.cursor() #Initialise the mycursor which will handle of the query executions

def printgenres(): #creation of a function to handle all of the select genres which are found in the code
    query = "SELECT genre from genre;"
    printrec(query)
    
def printrec(query): #creation of a function to handle all of the selects which print records
    mycursor.execute(query)
    retrieved_records= mycursor.fetchall()

    for record in retrieved_records:
        print(record)
    
#a series of prints which prints the choices for the user to choose from.
print("1) Insert Data")
print("2) Retrieve Data")
print("3) Update Record")
print("4) Delete Record")
print("5) Finish")
option = int(input("Enter your choice")) #a variable which takes the users choice
if option == 1: # an if statement is used to act on the choice of the user
    #A series of inputs and variables to gather the required data for the creation of a new record
    bud = str(input("Enter the budget:"))
    h_page = str(input("Enter the homepage"))
    lang_code = str(input("Enter the ISO code for the language"))
    o_title = str(input("Enter the Original Title"))
    overview = str(input("Enter the Overview"))
    pop = str(input("Enter the Popularity"))
    rel_date = str(input("Enter the Date it was or will be released YYYY-MM-DD"))
    rev = str(input("Enter the Revenue the movie has made"))
    time = str(input("Enter the runtime"))
    rel_state = str(input("Enter whether it is released or not"))
    tag = str(input("Enter the tagline of the movie"))
    en_title = str(input("Enter the English title of the movie"))
    avg_rating = str(input("Enter the average rating that the movie has been given"))
    rating_count = str(input("Enter the number of people who have rated the movie"))
    #Creates the sql which will be later filled with values
    mov_insert = "INSERT INTO movie (budget,homepage,original_language,original_title,overview,popularity," +\
    "release_date,revenue,runtime,release_status,tagline,title,vote_average,vote_count)" +\
    "VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
    #appends all of the users previous inputs into a variable
    vals = (bud,h_page,lang_code,o_title,overview,pop,rel_date,rev,time,rel_state,tag,en_title,avg_rating,rating_count)
    #executes the sql with the inserted variables
    mycursor.execute(mov_insert,vals)
    #commit ensures that the database commits to any changes made by the query
    mysql_connection.commit()
    #finds the last inserted record so that the user can see the record which was recently added.
    query = "SELECT LAST_INSERT_ID()"
    mycursor.execute(query)
    #gathers all records found by the query
    retrieved_records= mycursor.fetchall()
    #prints the records found
    for record in retrieved_records:
        print(record)
    #calls the printgenres function
    printgenres()
    #gathers the genre that the movie has
    genre = str(input("What Genre does the movie have? Case Sensitive"))
    #finds out whether the user wishes to enter a second genre
    option = str(input("Enter a second genre? y/n"))
    #if Yes
    if option == "y":
        #calls the printgenres function again
        printgenres()
        #takes another input
        genre2 = str(input("Enter the second genre:"))
        #creates the sql for the insert into hasgenre.
        geninsert = "INSERT INTO hasgenre VALUES ((SELECT id FROM movie where id = LAST_INSERT_ID())" +\
        ",(Select genre_id FROM genre WHERE genre = '"+genre+"'))," +\
        "((SELECT id FROM movie ORDER BY id DESC LIMIT 1)" +\
        ",(Select genre_id FROM genre WHERE genre = '"+genre2+"'));" 
        select = "SELECT Distinct movie.id,movie.title,movie.vote_average,genre.genre FROM movie " +\
        "INNER JOIN hasgenre ON movie.id = hasgenre.id " +\
        "INNER JOIN genre ON hasgenre.genre_id = genre.genre_id " +\
        "ORDER BY movie.id DESC LIMIT 2;"
    else:
        #creates the sql for the insert into hasgenre.
        geninsert = "INSERT INTO hasgenre VALUES ((SELECT id FROM movie where id = LAST_INSERT_ID())" +\
        ",(Select genre_id FROM genre WHERE genre = '"+genre+"'));" 
        select = "SELECT movie.id,movie.title,movie.vote_average,genre.genre FROM movie " +\
        "INNER JOIN hasgenre ON movie.id = hasgenre.id " +\
        "INNER JOIN genre ON hasgenre.genre_id = genre.genre_id " +\
        "ORDER BY movie.id DESC LIMIT 1;"
    mycursor.execute(geninsert)
    mysql_connection.commit()
    
    printrec(select)
elif option == 2:
    # series of prints for the user's choices
    print("1) ID")
    print("2) Original Title")
    print("3) English Title")
    print("4) Genre")
    print("5) Tagline")
    print("6) Quit")
    #takes the user's input
    field = int(input("What Field do you want to search with?"))
    if field == 1:
        #requires the user's input for what column value they are searching for
        search= str(input("Please enter the Movie ID:"))
        #creates the query based on the user's input
        query = "SELECT movie.id,movie.title,movie.vote_average FROM movie " +\
        "Where id = "+search+";"
        #calls the function which handles generic select queries
        printrec(query)
    elif field == 2:
        search= str(input("Please enter the Original Title of the Movie:"))
        query = "SELECT movie.id,movie.title,movie.vote_average FROM movie " +\
        "WHERE original_title LIKE '%"+search+"%';"
        printrec(query)
    elif field == 3:
        search= str(input("Please enter the English Title of the Movie:"))
        query = "SELECT movie.id,movie.title,movie.vote_average FROM movie " +\
        "WHERE title LIKE '%"+search+"%';"
        printrec(query)
    elif field == 4:
        printgenres()
        search= str(input("Please enter the Genre of the Movie from the above list:"))
        query = "SELECT movie.id,movie.title,movie.vote_average,genre.genre FROM movie " +\
        "INNER JOIN hasgenre on movie.id = hasgenre.id " +\
        "INNER JOIN genre ON hasgenre.genre_id = genre.genre_id " +\
        "WHERE genre.genre = '"+search+"';"
        printrec(query)
    elif field == 5:
        search= str(input("Please enter the tagline of the Movie:"))
        query = "SELECT movie.id,movie.title,movie.vote_average,keyword.keyword FROM movie " +\
        "INNER JOIN haskeyword on movie.id = haskeyword.id " +\
        "INNER JOIN keyword ON haskeyword.keyword_id = keyword.keyword_id " +\
        "WHERE keyword.keyword LIKE '%"+search+"%';"
        printrec(query)
elif option == 3:
    # code to update record
    query = "SELECT column_name FROM Information_Schema.COLUMNS " +\
        "WHERE table_name = 'movie';"
    #finds the ID of the movie that is to be updated
    rowval =str(input("What is the ID of the movie you'd like to update?"))
    select = "SELECT * FROM movie WHERE id = "+rowval+";"
    mycursor.execute(select)
    retrieved_records= mycursor.fetchall()
    if len(retrieved_records)<1: #detects if the user entered an ID which does not exist in the database
        print("No record found")
    else:
        for record in retrieved_records:
            print(record)
            
        printrec(query)
        #requests the user to enter which column is going to be updated
        col = str(input("What column would you like to update?"))
        #requests the suer to enter the new value of that column
        colval =str(input("What input would you like to update it with?"))
        #creates the sql required for the Update, col is the column which will be changed, colval is the new value of the column and rowval is the id of the record that is required to be changed.
        update="UPDATE movie SET "+col+"= '"+colval+"' WHERE id = "+rowval+";"
        mycursor.execute(update)
        mysql_connection.commit()
        #selects the changed record and the prints it
        select = "SELECT * FROM movie " +\
        "WHERE id = "+rowval+";"
        printrec(select)
elif option == 4:
    # code to delete record
        search= str(input("Please enter the Movie ID to delete a record:"))
        #the following is to delet the record in all tables that it may be referenced in, this is to avoid any errors when running the delete query
        movselect = "SELECT movie.id,movie.title,movie.vote_average FROM movie Where ID = "+search+";"
        movdelete = "DELETE FROM movie Where id = "+search+";"
        gendelete = "DELETE FROM hasgenre Where id = "+search+";"
        keydelete = "DELETE FROM haskeyword Where id = "+search+";"
        mycursor.execute(movselect)
        retrieved_records= mycursor.fetchall()
        for record in retrieved_records:
            print(record)
        print("Confirm deletion?")
        confirm = str(input("y/n"))
        #The last query to run is the main movie delete so that it will be delete with no referenced records.
        mycursor.execute(keydelete)
        mysql_connection.commit()
        mycursor.execute(gendelete)
        mysql_connection.commit()
        mycursor.execute(movdelete)
        mysql_connection.commit()
        print("Record has been deleted.")
        print("Program will now close")

1) Insert Data
2) Retrieve Data
3) Update Record
4) Delete Record
5) Finish
Enter your choice4
Please enter the Movie ID to delete a record:459536
(459536, 'New Title', 10.0)
Confirm deletion?
y/ny
Record has been deleted.
Program will now close


# 